# Network Analysis

Many interesting data problems can be fruitfully thought of in terms of **networks**, consisting of `nodes` of some type and the `edges` that join them.

For instance, your Facebook friends form the `nodes` of a network whose `edges` are friendship relations. A less obvious example is the World Wide Web itself, with each web page a `node` and each hyperlink from one page to another an `edge`.